# Generating names using bi-grams
We want to generate names in english based on a collection of existing english names.

In [ ]:
!head data/names.txt

In [ ]:
# store all the names in the file
words = open('data/names.txt', 'r').read().splitlines()
words[:10]

In [ ]:
len(words)

In [ ]:
# max / min size of names
max(len(w) for w in words), min(len(w) for w in words)

# Generating words using probabilities estimated using counting

In [ ]:
# bigrams
for w in words[:1]:
    print('--> ', w)
    for ch1, ch2 in zip(w, w[1:]):
        print(ch1, ch2)

In [ ]:
# add special element for starting and ending of words
for w in words[:3]:
    print('--> ', w)
    chs = ['<S>'] + list(w) + ['<E>']
    for ch1, ch2 in zip(chs, chs[1:]):
        print(ch1, ch2)

In [ ]:
# lets learn about the statistics of names, by counting
b = {}
for w in words[:10]:
    chs = ['<S>'] + list(w) + ['<E>']
    for ch1, ch2 in zip(chs, chs[1:]):
        bigram = ch1, ch2
        b[bigram] = b.get(bigram, 0) + 1

b

In [ ]:
# lets do it for all the words
b = {}
for w in words:
    chs = ['<S>'] + list(w) + ['<E>']
    for ch1, ch2 in zip(chs, chs[1:]):
        bigram = ch1, ch2
        b[bigram] = b.get(bigram, 0) + 1
b

In [ ]:
# lets sort by frequency 
sorted(b.items(), key=lambda x: x[1], reverse=True)[:10]

In [ ]:
# find all the characters
all_chars = ['<S>', '<E>'] + sorted(list(set("".join(words))))
print(all_chars)

In [ ]:
# mappings between number and encodings
itos = {idx: v for idx, v in enumerate(all_chars)}
itos

In [ ]:
stoi = {v: k for k, v in itos.items()}
stoi

In [ ]:
import torch

N = torch.zeros((len(all_chars), len(all_chars)), dtype=torch.int32)
for w in words:
    chs = ['<S>'] + list(w) + ['<E>']
    for ch1, ch2 in zip(chs, chs[1:]):
        N[stoi[ch1], stoi[ch2]] += 1
N

In [ ]:
# Lets visualize it better
import matplotlib.pyplot as plt
%matplotlib inline
plt.imshow(N)
plt.show()

In [ ]:
# Lets improve it a little
plt.figure(figsize=(16, 16))
plt.imshow(N, cmap='Blues')
for i in range(len(all_chars)):
    for j in range(len(all_chars)):
        chstr = itos[i] + itos[j]
        plt.text(j, i, chstr, ha='center', va='bottom', color='gray')
        plt.text(j, i, N[i,j].item(), ha='center', va='top', color='gray')
plt.axis('off')
plt.show()

In [ ]:
# Note that there is an empty row and an empty column, because start and end markers. 
# Lets use a single marker, which can be differentiated by the context.

all_chars = ['.'] + sorted(list(set("".join(words))))
itos = {idx: v for idx, v in enumerate(all_chars)}
stoi = {v: k for k, v in itos.items()}

N = torch.zeros((len(all_chars), len(all_chars)), dtype=torch.int32)
for w in words:
    chs = ['.'] + list(w) + ['.']
    for ch1, ch2 in zip(chs, chs[1:]):
        N[stoi[ch1], stoi[ch2]] += 1
        
# Lets improve it a little
plt.figure(figsize=(16, 16))
plt.imshow(N, cmap='Blues')
for i in range(len(all_chars)):
    for j in range(len(all_chars)):
        chstr = itos[i] + itos[j]
        plt.text(j, i, chstr, ha='center', va='bottom', color='gray')
        plt.text(j, i, N[i,j].item(), ha='center', va='top', color='gray')
plt.axis('off')
plt.show()

How can we use this matrix to generate names? 

In [ ]:
# Counts of the first name character
N[0,:]

In [ ]:
# Transforms to probabilities
p = N[0].float()
p = p / p.sum()
p

In [ ]:
# To sample from a distribution we will use torch.multinomial
a = torch.tensor([0.7, 0.2, 0.1])
sample = torch.multinomial(a, num_samples=100, replacement=True)
sample

In [ ]:
from collections import Counter
Counter(s.item() for s in sample)

In [ ]:
# Now sample from this distribution, using a deterministic generator
g = torch.Generator().manual_seed(2147483647)
ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g)
ix.item(), itos[ix.item()]

In [ ]:
# now lets find the second character of the generated word
p = N[13].float()
p = p / p.sum()
ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g)
ix.item(), itos[ix.item()]

In [ ]:
# and next one ..
p = N[9].float()
p = p / p.sum()
ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g)
ix.item(), itos[ix.item()]

In [ ]:
# lets put this into a cycle
ix = 0
while True:
    p = N[ix].float()
    p = p / p.sum()
    ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g)
    ix = ix.item()
    if ix == 0:
        break
    print(itos[ix])

In [ ]:
# generate a collection
g = torch.Generator().manual_seed(2147483647)
for _ in range(20):
    ix = 0
    result = []
    while True:
        p = N[ix].float()
        p = p / p.sum()
        ix = torch.multinomial(p,  num_samples=1, replacement=True, generator=g)
        ix = ix.item()
        if ix == 0:
            break
        result.append(itos[ix])
    print("".join(result))

In [ ]:
# One improvement of the code. Calculate the probability matrix instead of the count matrix
P = N.float() / N.sum(1, keepdim=True)


In [ ]:
P.sum(1)

In [ ]:
P[0]

In [ ]:
N[0] / N[0].sum()

In [ ]:
# generate a collection
g = torch.Generator().manual_seed(2147483647)
for _ in range(20):
    ix = 0
    result = []
    while True:
        p = P[ix].float()
        ix = torch.multinomial(p,  num_samples=1, replacement=True, generator=g)
        ix = ix.item()
        if ix == 0:
            break
        result.append(itos[ix])
    print("".join(result))

In [ ]:
# the results are not impressive, but it is different than a pure random generator.
for _ in range(20):
    result = []
    while True:
        p = torch.ones(len(all_chars))
        p = p / p.sum()
        ix = torch.multinomial(p,  num_samples=1, replacement=True, generator=g)
        ix = ix.item()
        if ix == 0:
            break
        result.append(itos[ix])
    print(''.join(result))

In any case, bigrams do not contains enough information about the names structure in order to generate 'real' ones.
- In this example, the probabilities are the parameters learn from the data

## Evaluating the model generated
Lets show the probabilities associated to each next character according to the target probability distribution (first three words):

In [ ]:
for w in words[:3]:
    chs = ['.'] + list(w) + ['.']
    for ch1, ch2 in zip(chs, chs[1:]):
        bigram = ch1, ch2
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        prob = P[ix1, ix2]
        print(f"{ch1}{ch2}: {prob:.4f}")

If all the characters were equaly probable, all the probabilities would be 1/27=0.04

You can see that many of these probabilities are above that value, and a few are below.

If we have a very good model, we expect the probabilities to be very close to 1, so the correct character will be very probable.

How to summarize these probabilities into a single number, so we can evaluate the behavior of the model?

We will use the likelihood, which in this case is the product of all the probabilities assigned by the model.
- This product will be near to 1 when all the probabilities are closer to 1

In [ ]:
likelihood = 1
for w in words[:3]:
    chs = ['.'] + list(w) + ['.']
    for ch1, ch2 in zip(chs, chs[1:]):
        bigram = ch1, ch2
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        prob = P[ix1, ix2]
        likelihood *= prob
likelihood

Since multiplying very small numbers will drop to zero very fast, we will use the logarithm (log-likehood). 
- Note that the logarithm function is monotonic

In [ ]:
for w in words[:3]:
    chs = ['.'] + list(w) + ['.']
    for ch1, ch2 in zip(chs, chs[1:]):
        bigram = ch1, ch2
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        prob = P[ix1, ix2]
        logprob = torch.log(prob)
        print(f"{ch1}{ch2}: {prob:.4f} {logprob:.4f}")

In [ ]:
log_likelihood = 0.0
n = 0
for w in words[:3]:
    chs = ['.'] + list(w) + ['.']
    for ch1, ch2 in zip(chs, chs[1:]):
        bigram = ch1, ch2
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        prob = P[ix1, ix2]
        n += 1
        logprob = torch.log(prob)
        log_likelihood += logprob
log_likelihood, (log_likelihood / n)

Since we are finding a loss function, we need lower values to be better, so we can multiply the average log-likelihood by -1.

In [ ]:
nll = -log_likelihood
nll, nll / n

In [ ]:
# Lets calculate the average nll in the whole dataset
log_likelihood = 0.0
n = 0
for w in words:
    chs = ['.'] + list(w) + ['.']
    for ch1, ch2 in zip(chs, chs[1:]):
        bigram = ch1, ch2
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        prob = P[ix1, ix2]
        n += 1
        logprob = torch.log(prob)
        log_likelihood += logprob
nll = -log_likelihood
nll / n

In [ ]:
# You can evaluate the nll in any word
for w in ['mary', 'joanne', 'balakrishnan', 'fakir', 'keshia', 'gavfed']:
    log_likelihood = 0.0
    n = 0
    chs = ['.'] + list(w) + ['.']
    for ch1, ch2 in zip(chs, chs[1:]):
        bigram = ch1, ch2
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        prob = P[ix1, ix2]
        n += 1
        logprob = torch.log(prob)
        log_likelihood += logprob
    nll = -log_likelihood
    print(w, nll/n)

The last result is due to the (v,f) bigram never appears in the dataset, so its probability is zero, and the logarithms is infinite.

In order to solve this problem, we can add one to all the counts in the counts matrix

In [ ]:
P = (N + 1).float() / N.sum(1, keepdim=True)

# You can evaluate the nll in any word
for w in ['mary', 'joanne', 'balakrishnan', 'fakir', 'keshia', 'gavfed']:
    log_likelihood = 0.0
    n = 0
    chs = ['.'] + list(w) + ['.']
    for ch1, ch2 in zip(chs, chs[1:]):
        bigram = ch1, ch2
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        prob = P[ix1, ix2]
        n += 1
        logprob = torch.log(prob)
        log_likelihood += logprob
    nll = -log_likelihood
    print(w, nll/n)

Resume:
- We train a model to generate names based on frequency counting
- We estimate probabilities using the counting, solving the zero-counts problems
- We develop a way to evaluate the quality of the model using the negative log-likehood
- We generate words based on the model. Words are not very good, but they are better than a pure random model.
